Imports

In [ ]:
import pyodbc
import pandas as pd

Csatlakozzunk egy már létező adatbázishoz

In [ ]:
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost\\SQLEXPRESS;"
    "Database=master;"
    "Trusted_Connection=yes;",
    autocommit=True # nem kell mindig beírni, hogy conn.commit() minden execute után
)


cursor = conn.cursor()

cursor.execute("USE master")

Hozzuk lére az 'autosiskola' adatbázist

In [ ]:
cursor.execute("IF EXISTS (SELECT name FROM sys.databases WHERE name = 'autosiskola') DROP DATABASE autosiskola")   ### újrafuttatás miatt
cursor.execute("CREATE DATABASE autosiskola")
cursor.execute("USE autosiskola")

conn.close()

Csatlakozás az 'autosiskola' adatbázishoz

In [ ]:
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost\\SQLEXPRESS;"
    "Database=autosiskola;"
    "Trusted_Connection=yes;",
    autocommit=True
)
cursor = conn.cursor()

Töröljük a táblákat, amennyiben léteznek (újrafuttatása miatt)

In [ ]:
drop_tables_sql = """
DROP TABLE IF EXISTS Lessons;
DROP TABLE IF EXISTS Exams;
DROP TABLE IF EXISTS Vehicles;
DROP TABLE IF EXISTS Students;
DROP TABLE IF EXISTS Instructors;
"""

for stmt in drop_tables_sql.strip().split(";"):
    if stmt.strip():
        cursor.execute(stmt)


Hozzuk létre a táblákat

In [ ]:
create_tables_sql = """
CREATE TABLE Students (
    tanulo_id INT PRIMARY KEY,
    nev NVARCHAR(100) NOT NULL,
    szuletesi_datum DATE NOT NULL,
    lakcim NVARCHAR(200) NOT NULL,
    telefonszam NVARCHAR(30),
    email NVARCHAR(100),
    jelentkezes_datum DATE NOT NULL
);

CREATE TABLE Instructors (
    oktato_id INT PRIMARY KEY,
    nev NVARCHAR(100) NOT NULL,
    jogositvany_tipus CHAR(1) NOT NULL CHECK (jogositvany_tipus IN ('A', 'B', 'C', 'D')),
    munkaviszony_kezdete DATE NOT NULL,
    telefonszam NVARCHAR(30),
    email NVARCHAR(100)
);

CREATE TABLE Lessons (
    ora_id INT PRIMARY KEY,
    tanulo_id INT NOT NULL,
    oktato_id INT NOT NULL,
    datum DATE NOT NULL,
    idopont TIME NOT NULL,
    ora_tipus NVARCHAR(20) NOT NULL CHECK (ora_tipus IN ('elméleti', 'gyakorlati')),
    helyszin NVARCHAR(100),
    FOREIGN KEY (tanulo_id) REFERENCES Students(tanulo_id),
    FOREIGN KEY (oktato_id) REFERENCES Instructors(oktato_id)
);

CREATE TABLE Vehicles (
    jarmu_id INT PRIMARY KEY,
    rendszam NVARCHAR(20) NOT NULL UNIQUE,
    marka NVARCHAR(50),
    tipus NVARCHAR(50),
    evjarat INT CHECK (evjarat >= 1990),
    muszaki_ervenyesseg DATE,
    oktato_id INT NOT NULL,
    FOREIGN KEY (oktato_id) REFERENCES Instructors(oktato_id)
);

CREATE TABLE Exams (
    vizsga_id INT PRIMARY KEY,
    tanulo_id INT NOT NULL,
    vizsga_tipus NVARCHAR(20) NOT NULL CHECK (vizsga_tipus IN ('elméleti', 'forgalmi')),
    datum DATE NOT NULL,
    eredmeny NVARCHAR(20) NOT NULL CHECK (eredmeny IN ('sikeres', 'sikertelen')),
    FOREIGN KEY (tanulo_id) REFERENCES Students(tanulo_id)
);
"""

# a cursor.execute egyszerre csak 1 utasítást tud végrehajtani, ezért végig kell iterálni a create_table_sql-en

for statement in create_tables_sql.strip().split(";"):
    if statement.strip():
        cursor.execute(statement)



Táblák feltöltése adatokkal

In [ ]:
### Fájlok beolvasása
students = pd.read_csv("students.csv", encoding="utf-8")
instructors = pd.read_csv("instructors.csv", encoding="utf-8")
lessons = pd.read_csv("lessons.csv", encoding="utf-8")
vehicles = pd.read_csv("vehicles.csv", encoding="utf-8")
exams = pd.read_csv("exams.csv", encoding="utf-8")


### Táblák feltöltées executemanyvel

students_insert_sql = """
INSERT INTO Students (tanulo_id, nev, szuletesi_datum, lakcim, telefonszam, email, jelentkezes_datum)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
students_adatok = students.values.tolist()
cursor.executemany(students_insert_sql, students_adatok)

instructors_insert_sql = """
INSERT INTO Instructors (oktato_id, nev, jogositvany_tipus, munkaviszony_kezdete, telefonszam, email)
VALUES (?, ?, ?, ?, ?, ?)
"""
instructors_adatok = instructors.values.tolist()
cursor.executemany(instructors_insert_sql, instructors_adatok)

lessons_insert_sql = """
INSERT INTO Lessons (ora_id, tanulo_id, oktato_id, datum, idopont, ora_tipus, helyszin)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
lessons_adatok = lessons.values.tolist()
cursor.executemany(lessons_insert_sql, lessons_adatok)

vehicles_insert_sql = """
INSERT INTO Vehicles (jarmu_id, rendszam, marka, tipus, evjarat, muszaki_ervenyesseg, oktato_id)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""
vehicles_adatok = vehicles.values.tolist()
cursor.executemany(vehicles_insert_sql, vehicles_adatok)

exams_insert_sql = """
INSERT INTO Exams (vizsga_id, tanulo_id, vizsga_tipus, datum, eredmeny)
VALUES (?, ?, ?, ?, ?)
"""
exams_adatok = exams.values.tolist()
cursor.executemany(exams_insert_sql, exams_adatok)

7 lekérdezés és eredményeik (egyszerűbbektől az összetettekig)

In [ ]:
sql_1 = """
SELECT nev, szuletesi_datum FROM Students ORDER BY szuletesi_datum
"""
cursor.execute(sql_1)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_1_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_1_df


In [ ]:
sql_2 = """
SELECT nev FROM Students WHERE YEAR(szuletesi_datum) > 1990
"""
cursor.execute(sql_2)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_2_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_2_df


In [ ]:
sql_3 = """
SELECT S.nev AS tanulo_nev, I.nev AS oktato_nev
FROM Lessons L
JOIN Students S ON L.tanulo_id = S.tanulo_id
JOIN Instructors I ON L.oktato_id = I.oktato_id
"""
cursor.execute(sql_3)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_3_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_3_df


In [ ]:
sql_4 = """
SELECT I.nev, COUNT(DISTINCT L.tanulo_id) AS tanulok_szama
FROM Lessons L
JOIN Instructors I ON L.oktato_id = I.oktato_id
GROUP BY I.nev
"""
cursor.execute(sql_4)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_4_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_4_df


In [ ]:
sql_5 = """
SELECT I.nev, COUNT(DISTINCT L.tanulo_id) AS tanulok_szama
FROM Lessons L
JOIN Instructors I ON L.oktato_id = I.oktato_id
GROUP BY I.nev
HAVING COUNT(DISTINCT L.tanulo_id) >= 2
"""
cursor.execute(sql_5)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_5_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_5_df


In [ ]:
sql_6 = """
SELECT ora_tipus, COUNT(*) AS ora_db
FROM Lessons
GROUP BY ROLLUP (ora_tipus)
"""
cursor.execute(sql_6)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_6_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_6_df


In [ ]:
sql_7 = """
SELECT nev, szuletesi_datum,
       RANK() OVER (ORDER BY szuletesi_datum) AS szuletesi_sorrend
FROM Students
"""
cursor.execute(sql_7)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

lekerdezes_7_df = pd.DataFrame(sorok, columns=oszlopok)
lekerdezes_7_df


7 CRUD művelet (egyszerűbbektől az összetettekig)

Create

In [ ]:
sql_C1 = """
INSERT INTO Students (tanulo_id, nev, szuletesi_datum, lakcim, telefonszam, email, jelentkezes_datum)
VALUES (999, 'Teszt Elek', '2000-01-01', 'Teszt utca 1.', '0600000000', 'teszt@pelda.hu', '2025-01-01')
"""
cursor.execute(sql_C1)


Read

In [ ]:
sql_C2 = "SELECT * FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C2)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

CRUD_2_df = pd.DataFrame(sorok, columns=oszlopok)
CRUD_2_df


Update

In [ ]:
sql_C3 = """
UPDATE Students SET lakcim = 'Módosított utca 99.' WHERE tanulo_id = 999
"""
cursor.execute(sql_C3)

Read - Update-elt lakcím ellenörzése

In [ ]:
sql_C4 = "SELECT nev, lakcim FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C4)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

CRUD_4_df = pd.DataFrame(sorok, columns=oszlopok)
CRUD_4_df


Delete

In [ ]:
sql_C5 = "DELETE FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C5)


Read - Delete ellenőrzése

In [ ]:
sql_C6 = "SELECT * FROM Students WHERE tanulo_id = 999"
cursor.execute(sql_C6)
rows = cursor.fetchall()
sorok = [tuple(row) for row in rows]
oszlopok = [desc[0] for desc in cursor.description]

CRUD_6_df = pd.DataFrame(sorok, columns=oszlopok)
CRUD_6_df


In [ ]:
sql_C7 = """
IF EXISTS (SELECT 1 FROM Students WHERE tanulo_id = 1)
   AND EXISTS (SELECT 1 FROM Instructors WHERE oktato_id = 1)
BEGIN
    INSERT INTO Lessons (ora_id, tanulo_id, oktato_id, datum, idopont, ora_tipus, helyszin)
    VALUES (9999, 1, 1, '2025-05-16', '08:00', 'gyakorlati', 'Központi gyakorlópálya')
END
"""
cursor.execute(sql_C7)


In [ ]:
conn.close()